In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
EPOCHS = 5
YEAR = '2000'
#YEAR = '2010'

df = pd.read_csv('./data/census/census_%s.csv' % YEAR)
df.dropna(subset=['name'], inplace=True)
df.replace('(S)', 0, inplace=True)
df

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,SMITH,1,2376206,880.85,880.85,73.35,22.22,0.40,0.85,1.63,1.56
1,JOHNSON,2,1857160,688.44,1569.30,61.55,33.80,0.42,0.91,1.82,1.50
2,WILLIAMS,3,1534042,568.66,2137.96,48.52,46.72,0.37,0.78,2.01,1.60
3,BROWN,4,1380145,511.62,2649.58,60.71,34.54,0.41,0.83,1.86,1.64
4,JONES,5,1362755,505.17,3154.75,57.69,37.73,0.35,0.94,1.85,1.44
5,MILLER,6,1127803,418.07,3572.82,85.81,10.41,0.42,0.63,1.31,1.43
6,DAVIS,7,1072335,397.51,3970.33,64.73,30.77,0.40,0.79,1.73,1.58
7,GARCIA,8,858289,318.17,4288.50,6.17,0.49,1.43,0.58,0.51,90.81
8,RODRIGUEZ,9,804240,298.13,4586.62,5.52,0.54,0.58,0.24,0.41,92.70
9,WILSON,10,783051,290.27,4876.90,69.72,25.32,0.46,1.03,1.74,1.73


## Resampling with weight

In [2]:
sdf = df.sample(1000000, weights=df['count'], replace=True)

## Assign race by pertcentage

In [3]:
from numpy.random import choice

races = ['white', 'black', 'api', 'hispanic']

def to_race(c):
    w = np.array(c).astype(float)
    probs = w/w.sum()
    return choice(races, p=probs)

sdf['race'] = sdf[['pctwhite', 'pctblack', 'pctapi', 'pcthispanic']].apply(lambda c: to_race(c), axis=1)
sdf

/opt/venv/lib/python2.7/site-packages/ipykernel/__main__.py:7: RuntimeWarning: invalid value encountered in divide
/opt/venv/lib/python2.7/site-packages/ipykernel/__main__.py:8: RuntimeWarning: invalid value encountered in less


,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic,race
1817,MEDEIROS,1818,18121,6.72,47759.96,85.14,0.33,3.06,0.14,7.27,4.07,white
26179,NOWLAN,26163,879,0.33,77509.45,95.22,0,0.91,0,1.82,1.02,white
121,STEVENS,122,181417,67.25,18684.77,83.40,11.40,0.53,1.23,1.64,1.79,white
1193,WOMACK,1194,26890,9.97,42674.71,68.30,27.53,0.33,0.71,1.72,1.40,white
49,TORRES,50,325169,120.54,12433.80,6.05,0.58,1.42,0.26,0.53,91.16,hispanic
31161,SKAFF,31126,705,0.26,78960.22,95.32,0.71,0,0,1.84,1.84,white
698,MOSLEY,699,44698,16.57,36382.71,42.69,52.83,0.26,0.53,2.18,1.51,white
21144,BRODE,21142,1157,0.43,75630.80,96.02,1.12,0,0,0.61,1.38,white
44726,TROTTI,44654,453,0.17,81777.92,96.69,0,0.00,0,1.32,0,white
12377,FUJII,12377,2302,0.85,70365.91,5.82,0,84.40,0,7.04,2.13,api


## Check the correctness of race assignment

In [4]:
df[df.name == 'SMITH']

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,SMITH,1,2376206,880.85,880.85,73.35,22.22,0.40,0.85,1.63,1.56


In [5]:
xdf = sdf[sdf.name=='SMITH'].groupby(['race']).agg({'name': 'count'})
xdf * 100 / xdf.sum()

,name
race,
api,0.525465
black,22.564673
hispanic,1.546079
white,75.363783


In [6]:
# Additional features
sdf['name_last'] = sdf.name.str.title()
sdf.groupby('race').agg({'name_last': 'count'})

,name_last
race,
api,34970
black,125879
hispanic,128146
white,711005


In [7]:
len(sdf)

1000000

## Preprocessing the input data

In [8]:
# only last name in Census data
sdf['name_last_name_first'] = sdf['name_last']

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
#vect = CountVectorizer(analyzer='char', ngram_range=(2, 2), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_
len(vocab)

963

In [9]:
import operator
sorted_vocab = sorted(vocab.items(), key=operator.itemgetter(1))
cols = list(map(operator.itemgetter(0), sorted_vocab))

In [10]:
count_df = pd.DataFrame(a.todense(), columns=cols)
count_df

,Aa,Ab,Ac,Ad,Ae,Af,Ag,Ah,Ai,Aj,...,zp,zq,zr,zs,zt,zu,zv,zw,zy,zz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
count_df.sum().sort_values(ascending=False).describe()

count       963.000000
mean       5508.892004
std       12662.615933
min           3.000000
25%          79.500000
50%         911.000000
75%        5417.500000
max      175908.000000
dtype: float64

In [12]:
pd.set_option('display.max_rows', 20)
count_df.sum().sort_values(ascending=False)

er    175908
on    124264
an    106667
ar    100173
ll     86314
in     82185
le     73190
en     67003
el     61376
so     57480
       ...  
Gf         3
kv         3
fv         3
Vn         3
Tc         3
Gd         3
xv         3
gj         3
Gm         3
Iu         3
dtype: int64

In [13]:
# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = [w[1] for w in words]
num_words = len(words_list)
print("num_words = %d" % num_words)


def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

num_words = 963
Max feature len = 14, Avg. feature len = 5


In [14]:
len(vocab)

963

## Train a LSTM model

ref: http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [15]:
'''The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model

max_features = num_words # 20000
feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Using TensorFlow backend.


800000 train sequences
200000 test sequences
Pad sequences (samples x time)
X_train shape: (800000, 20)
X_test shape: (200000, 20)
4 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (800000, 4)
y_test shape: (200000, 4)


In [16]:
print('Build model...')

model = Sequential()
model.add(Embedding(num_words, 32, input_length=feature_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 32)            30816     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 516       
Total params: 113,764.0
Trainable params: 113,764
Non-trainable params: 0.0
_________________________________________________________________
None


In [17]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=2)
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 720000 samples, validate on 80000 samples
Epoch 1/5
1214s - loss: 0.5751 - acc: 0.8031 - val_loss: 0.5465 - val_acc: 0.8114
Epoch 2/5
1277s - loss: 0.5359 - acc: 0.8152 - val_loss: 0.5269 - val_acc: 0.8173
Epoch 3/5
1168s - loss: 0.5233 - acc: 0.8185 - val_loss: 0.5173 - val_acc: 0.8192
Epoch 4/5
1236s - loss: 0.5159 - acc: 0.8205 - val_loss: 0.5127 - val_acc: 0.8216
Epoch 5/5
1181s - loss: 0.5110 - acc: 0.8222 - val_loss: 0.5077 - val_acc: 0.8234
Test score: 0.50193548161
Test accuracy: 0.82522


```
Train...
Train on 72000 samples, validate on 8000 samples
Epoch 1/5
124s - loss: 0.6793 - acc: 0.7678 - val_loss: 0.6147 - val_acc: 0.7917
Epoch 2/5
156s - loss: 0.5908 - acc: 0.7979 - val_loss: 0.5909 - val_acc: 0.8001
Epoch 3/5
157s - loss: 0.5725 - acc: 0.8043 - val_loss: 0.5792 - val_acc: 0.8031
Epoch 4/5
206s - loss: 0.5621 - acc: 0.8071 - val_loss: 0.5726 - val_acc: 0.8039
Epoch 5/5
156s - loss: 0.5556 - acc: 0.8087 - val_loss: 0.5741 - val_acc: 0.8044
Test score: 0.56178010149
Test accuracy: 0.80655
```

## Confusion Matrix

In [18]:
y_pred = model.predict_classes(X_test, verbose=2)
p = model.predict_proba(X_test, verbose=2) # to predict probability
target_names = list(sdf.race.astype('category').cat.categories)
print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

             precision    recall  f1-score   support

        api       0.84      0.64      0.73      6994
      black       0.63      0.02      0.04     25176
   hispanic       0.86      0.83      0.85     25629
      white       0.82      0.98      0.89    142201

avg / total       0.80      0.83      0.77    200000

[[  4507      5    685   1797]
 [   160    472    179  24365]
 [    94     13  21387   4135]
 [   581    254   2688 138678]]


## Save model

In [19]:
model.save('./models/census/lstm/census%s_ln_lstm.h5' % YEAR)

In [20]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('./models/census/lstm/census%s_ln_vocab.csv' % YEAR, index=False, encoding='utf-8')